In [2]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [3]:

from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



# 🧹 Nettoyage de la table usagers 


In [4]:
# Etablir connection avec la base de données
engine = get_engine(DB_URL) 

In [5]:
df_usagers = get_df_from_table(engine, "bronze", "usagers_raw")
df_usagers.head()

,identifiant_de_l_accident,identifiant_vehicule,place,categorie_d_usager,gravite,sexe,annee_de_naissance,motif_trajet,existence_equipement_de_securite,utilisation_equipement_de_securite,localisation_du_pieton,action_pieton,pieton_seul_ou_non
0,201700033619,"B01,A01","1,1,2,2","Conducteur,Conducteur,Passager,Passager","Blessé,Blessé,Blessé,Blessé","Masculin,Masculin,Masculin,Masculin","1996,1939,1996,1931","Promenade – loisirs,Promenade – loisirs,Promen...","Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
1,201700048262,"A01,E01,C01,B01,D01","1,1,1,1,1","Conducteur,Conducteur,Conducteur,Conducteur,Co...","Indemne,Indemne,Blessé,Indemne,Indemne","Masculin,Masculin,Féminin,Féminin,Masculin","1952,1934,1979,1986,1978",None,"Ceinture,Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
2,201700048288,"D01,B01,A01,C01","1,1,1,2,1","Conducteur,Conducteur,Conducteur,Passager,Cond...","Indemne,Indemne,Indemne,Blessé,Indemne","Masculin,Masculin,Masculin,Masculin,Masculin","1982,1978,1990,2004,1978",None,"Ceinture,Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
3,201700048486,"A01,B01","1,1,2","Conducteur,Conducteur,Passager","Blessé,Indemne,Blessé","Masculin,Masculin,Masculin","1992,1990,1992",Autre,"Ceinture,Ceinture,Ceinture","Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant",None
4,201700048639,"Z01,A01","1,1","Conducteur,Conducteur","Indemne,Blessé","Masculin,Masculin","1992,1960","Domicile – travail,Domicile – travail","Ceinture,Casque","Oui,Oui",None,"Se déplaçant,Se déplaçant",None


In [6]:
# Supprimer les doublons
print(f"Nombre de lignes avant suppression des doublons : {df_usagers.shape[0]}")
df_usagers = df_usagers.drop_duplicates()
print(f"Nombre de lignes après suppression des doublons : {df_usagers.shape[0]}")
#remplacer les valeurs nulles
remplacer_valeurs_nulles(df_usagers)
#Nettoyage sémantique
#Traiter valeurs spéciales (-1, 00, NULL chaîne).
df_usagers =  traiter_valeurs_speciales(df_usagers)
df_usagers.head()

Nombre de lignes avant suppression des doublons : 1427733
Nombre de lignes après suppression des doublons : 475911


,identifiant_de_l_accident,identifiant_vehicule,place,categorie_d_usager,gravite,sexe,annee_de_naissance,motif_trajet,existence_equipement_de_securite,utilisation_equipement_de_securite,localisation_du_pieton,action_pieton,pieton_seul_ou_non
0,201700033619,"B01,A01","1,1,2,2","Conducteur,Conducteur,Passager,Passager","Blessé,Blessé,Blessé,Blessé","Masculin,Masculin,Masculin,Masculin","1996,1939,1996,1931","Promenade – loisirs,Promenade – loisirs,Promen...","Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
1,201700048262,"A01,E01,C01,B01,D01","1,1,1,1,1","Conducteur,Conducteur,Conducteur,Conducteur,Co...","Indemne,Indemne,Blessé,Indemne,Indemne","Masculin,Masculin,Féminin,Féminin,Masculin","1952,1934,1979,1986,1978",None,"Ceinture,Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
2,201700048288,"D01,B01,A01,C01","1,1,1,2,1","Conducteur,Conducteur,Conducteur,Passager,Cond...","Indemne,Indemne,Indemne,Blessé,Indemne","Masculin,Masculin,Masculin,Masculin,Masculin","1982,1978,1990,2004,1978",None,"Ceinture,Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant,Se dépl...",None
3,201700048486,"A01,B01","1,1,2","Conducteur,Conducteur,Passager","Blessé,Indemne,Blessé","Masculin,Masculin,Masculin","1992,1990,1992",Autre,"Ceinture,Ceinture,Ceinture","Oui,Oui,Oui",None,"Se déplaçant,Se déplaçant,Se déplaçant",None
4,201700048639,"Z01,A01","1,1","Conducteur,Conducteur","Indemne,Blessé","Masculin,Masculin","1992,1960","Domicile – travail,Domicile – travail","Ceinture,Casque","Oui,Oui",None,"Se déplaçant,Se déplaçant",None


In [7]:
#supprimer les colonnes avec missing values supérieures à 50%
col_a_supprimer = ['localisation_du_pieton', 'pieton_seul_ou_non']
df_usagers = supprimer_colonnes(df_usagers, col_a_supprimer)

In [8]:
#Harmonisation de la colonne 'sexe'
# prend les 2 premiers caractères
df_usagers['sexe'] = df_usagers['sexe'].astype(str).str[:2].str.lower().str.replace(' ', '')  
df_usagers.groupby('sexe').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
mapping_sexe = {
    'ma': 'homme',
    'fé': 'femme'
}
harmonise_colonne_df(df_usagers,'sexe',mapping_sexe)
print('\napres harmonisation:')
print(df_usagers.groupby('sexe').size().reset_index(name='counts').sort_values(by='counts', ascending=False))


apres harmonisation:
    sexe  counts
1  homme  332112
0  femme  143799


In [9]:
df_usagers.head(1)

,identifiant_de_l_accident,identifiant_vehicule,place,categorie_d_usager,gravite,sexe,annee_de_naissance,motif_trajet,existence_equipement_de_securite,utilisation_equipement_de_securite,action_pieton
0,201700033619,"B01,A01","1,1,2,2","Conducteur,Conducteur,Passager,Passager","Blessé,Blessé,Blessé,Blessé",homme,"1996,1939,1996,1931","Promenade – loisirs,Promenade – loisirs,Promen...","Ceinture,Ceinture,Ceinture,Ceinture","Oui,Oui,Oui,Oui","Se déplaçant,Se déplaçant,Se déplaçant,Se dépl..."


In [10]:
# df_usagers.groupby('categorie_d_usager').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('gravite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('gravite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('motif_trajet').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('motif_trajet').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('existence_equipement_de_securite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('utilisation_equipement_de_securite').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
# df_usagers.groupby('action_pieton').size().reset_index(name='counts').sort_values(by='counts', ascending=False)


In [11]:
#Fermeture des connexions actives à la base de données
admin_engine = get_engine(ADMIN_DB_URL)
show_open_connections(admin_engine, DB_NAME)
close_all_connections(admin_engine, DB_NAME)
show_open_connections(admin_engine, DB_NAME)



INFO:__main__:Toutes les connexions à 'db_accident' ont été fermées (sauf la session actuelle).
INFO:__main__:Engine SQLAlchemy libéré — plus aucune connexion active.


(3697, 'postgres', 'db_accident', 'idle', 'ROLLBACK')
